In [1]:
# ! pip install datasets soundfile librosa

## **Loading the Audio File**

In [2]:
import librosa

audio, sr = librosa.load("esc50_samples/sample_0.wav", sr=None)

print(sr)

44100


If you do not use sr=None, librosa will silently resample for you.

In [3]:
from IPython.display import Audio

audio, sr = librosa.load("esc50_samples/sample_0.wav", sr=None)

display(Audio(data=audio, rate=sr))

In [4]:
audio.shape

(220500,)

## **Audio Classification**

In [5]:
# Import pipeline
from transformers import pipeline
import torch

# Specify the inference task
classifier = pipeline(
    task="audio-classification",
    model="MIT/ast-finetuned-audioset-10-10-0.4593",
    dtype=torch.bfloat16,
)

/Users/kanavbansal/Developer/.env_jupyter/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Device set to use mps:0


In [9]:
audio, sr = librosa.load("esc50_samples/sample_0.wav", sr=None)

classifier(audio)[:5]

[{'score': 0.2844504117965698, 'label': 'Sigh'},
 {'score': 0.27291613817214966, 'label': 'Silence'},
 {'score': 0.10907381027936935, 'label': 'Roar'},
 {'score': 0.06132941320538521, 'label': 'Sound effect'},
 {'score': 0.04192547872662544, 'label': 'Groan'}]

In [10]:
classifier.feature_extractor.sampling_rate

16000

In [12]:
import librosa

def load_and_resample(path, target_sr=16000):
    audio, sr = librosa.load(path, sr=None)

    if sr != target_sr:
        audio = librosa.resample(
            y=audio,
            orig_sr=sr,
            target_sr=target_sr
        )

    return audio

In [13]:
audio = load_and_resample("esc50_samples/sample_0.wav", target_sr=classifier.feature_extractor.sampling_rate)

classifier(audio)[:5]

[{'score': 0.18755587935447693, 'label': 'Bark'},
 {'score': 0.17971062660217285, 'label': 'Animal'},
 {'score': 0.09851495921611786, 'label': 'Dog'},
 {'score': 0.08364809304475784, 'label': 'Bow-wow'},
 {'score': 0.07124319672584534, 'label': 'Domestic animals, pets'}]

## **Zero Shot Audio Classification**

Zero shot audio classification pipeline using `ClapModel`. This pipeline predicts the class of an audio when you provide an audio and a set of `candidate_labels`.

classifier(audio, candidate_labels=["Sound of a dog", "Sound of vaccum cleaner"])

CLAP model - Contrastive Language-Audio Pretraining

CLAP takes both Audio and Text as input and computes the similarity between the two. 



In [14]:
# Import pipeline
from transformers import pipeline
import torch

# Specify the inference task
zero_shot_classifier = pipeline(
    task="zero-shot-audio-classification",
    model="laion/clap-htsat-fused",
)

Device set to use mps:0


In [15]:
zero_shot_classifier.feature_extractor.sampling_rate

48000

In [16]:
audio = load_and_resample("esc50_samples/sample_0.wav", target_sr=zero_shot_classifier.feature_extractor.sampling_rate)

zero_shot_classifier(audio, candidate_labels=["sound of a dog", "sound of a vaccum"])

[{'score': 0.9999997615814209, 'label': 'sound of a dog'},
 {'score': 2.6279039389009995e-07, 'label': 'sound of a vaccum'}]

### **Important**

The input example should have the sampling rate that the model expects. 

- A soundwave is a continuous signal. Meaning it contains infinite values of signal value in a given time.
- But the audio your computer can work with is a series of discrete values known as Digital representation.
- To get the digital representation of a continuous soundwave, we first capture the sound with a microphone which helps convert it from Analog Signal to Electrical Signal.
- Then the electrical signal is sampled to get Digital Representation.

A very important characteristics of a Digital Audio is the Sampling Rate. It represents the number of samples in one second. It is measured in herts or kiloherts.
- At 8000 Hz or 8 kHz => A 5 second Audio Sample will have 40,000 array of signal values
- At 16000 Hz or 16 kHz => Same 5 sec Audio will have 80,000 signal values represented in an array


## **Automatic Speech Recognition**

Applications:
- Meeting Notes
- Auto Generated Close Captions

In [25]:
from IPython.display import Audio

audio, sr = librosa.load("librispeech_samples/sample_0.wav", sr=None)

display(Audio(data=audio, rate=sr))

In [26]:
print(sr)

16000


In [27]:
print(audio.shape)

(232480,)


In [28]:
from transformers import pipeline

asr = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-small",
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use mps:0


In [30]:
asr.feature_extractor.sampling_rate

16000

In [33]:
asr(audio)

{'text': ' Chapter 16. I might have told you of the beginning of this liaison in a few lines, but I wanted you to see every step by which we came. I, too, agree to whatever Marguerite wished.'}

## **Transcribing Long Audio Files**

Split the long audio file into smaller chunks. 

Each chunks size can be specified. For Whisper - 30 sec should be the chunk size. Make sure each segment to have a small amount of overlap with the previous segment. This allows the pipeline to accurately stitch the segment together at the boundaries and merge the transcriptions accordingly.  

Since the audio is split into chunks, the pipeline can transcribe each chunk independently and then combine the results. For this reason you can transcribe batches of chunks in parallel. 

## **Differnce between Stereo vs Mono Channel**

Most transformers work with Mono Channel.

In [ ]:
# audio.shape

# import numpy as np

# audio_transposed = np.transpose(audio)

# import librosa

# audio_mono = librosa.to_mono(audio_transposed)

In [ ]:
asr(
    audio,
    chunk_length_s=30,
    batch_size=4,
    return_timestamps=True,
)

## **Text To Speech**

In [34]:
from transformers import pipeline

narrator = pipeline(
    task="text-to-speech",
    model="suno/bark-small",
)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

speaker_embeddings_path.json: 0.00B [00:00, ?B/s]

Device set to use mps:0


In [40]:
text = """
Hello, my name is Kanav. And, uh — and I like pizza. [laughs] 
"""

In [41]:
narrated_text = narrator(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [43]:
print(narrated_text["sampling_rate"])

24000


In [42]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(
    narrated_text["audio"][0],
    rate=narrated_text["sampling_rate"]
)